In [2]:
import os
import re
import pandas as pd
from pathlib import Path

In [3]:

# Configuration

# attack tool
# log_dir = Path("/Users/justin/BDAA/ACL/code/agentdojo/logs/base-gemini")
# log_dir = Path("/Users/justin/BDAA/ACL/code/agentdojo/logs/base-qwen")
# log_dir = Path("/Users/justin/BDAA/ACL/code/agentdojo/logs/spotlight-qwen")
# log_dir = Path("/Users/justin/BDAA/ACL/code/agentdojo/logs/spotlight-gemini")
# log_dir = Path("/Users/justin/BDAA/ACL/code/agentdojo/logs/tool_filter_qwen")
# log_dir = Path("/Users/justin/BDAA/ACL/code/agentdojo/logs/tool_filter_gemini")
# log_dir = Path("/Users/justin/BDAA/ACL/code/agentdojo/logs/melon-qwen")

# log_dir = Path("/Users/justin/BDAA/ACL/code/agentdojo/logs/melon-gemini")
# log_dir = Path("/Users/justin/BDAA/ACL/code/agentdojo/logs/base-qwen")
# log_dir = Path("/Users/justin/BDAA/ACL/code/agentdojo/logs/base-gemini")
# log_dir = Path("/Users/justin/BDAA/ACL/code/agentdojo/logs/detector-qwen")
# log_dir = Path("/Users/justin/BDAA/ACL/code/agentdojo/logs/detector-gemini")
# log_dir = Path("/Users/justin/BDAA/ACL/code/agentdojo/logs/type_iii_b")


log_dir = Path("/Users/justin/BDAA/ACL/code/agentdojo/logs/detection_data_attack")
log_files = sorted(log_dir.glob("*.log"))

# Determine the framework type from log_dir name
if "console_melon_logs" in str(log_dir):
    framework = "melon"
elif "console_logs" in str(log_dir):
    framework = "base"
elif "progent-llm-generate-policy" in str(log_dir):
    framework = "progent"
elif "spotlight" in str(log_dir):
    framework = "spotlight"
else:
    framework = "unknown"

print(f"Analyzing logs from: {log_dir}")
print(f"Framework detected: {framework}")
print(f"Total log files found: {len(log_files)}")

Analyzing logs from: /Users/justin/BDAA/ACL/code/agentdojo/logs/detection_data_attack
Framework detected: unknown
Total log files found: 4


In [5]:
def extract_statistics(file_path):
    """
    Extract utility_true_count and security_true_ratio from log file.
    Returns tuple: (utility_count, utility_total, utility_ratio, security_count, security_total, security_ratio)
    """
    with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
        content = f.read()
    
    # Find the last occurrence of utility_true_count
    utility_pattern = r'utility_true_count=\((\d+)/(\d+)=([\d.]+)\)'
    utility_matches = re.findall(utility_pattern, content)
    
    # Find the last occurrence of security_true_ratio
    security_pattern = r'security_true_ratio=\((\d+)/(\d+)=([\d.]+)\)'
    security_matches = re.findall(security_pattern, content)
    
    if utility_matches and security_matches:
        # Get the last match
        utility_count, utility_total, utility_ratio = utility_matches[-1]
        security_count, security_total, security_ratio = security_matches[-1]
        
        return (
            int(utility_count),
            int(utility_total),
            float(utility_ratio),
            int(security_count),
            int(security_total),
            float(security_ratio)
        )
    
    return None


# Parse all log files

results = []
result_utility_attackType = {}
result_utility_ratio_attackType = {}

result_security_attackType = {}
result_security_ratio_attackType = {}
attack_cnt = {}
print(f"Processing {len(log_files)} log files...")
for log_file in log_files:
    file_name = log_file.name
    
    # Extract suite and attack type from filename
    # Format: {suite}_type_{attack_type}.log
    parts = file_name.replace('.log', '').split('_')
    # print(f"Processing file: {file_name} with parts: {parts}")
    if len(parts) == 2:
        suite = parts[0]
        
        if parts[1].startswith('slack'):
            attack_type = 'slack'
        elif parts[1].startswith('banking'):
            attack_type = 'banking'
        elif parts[1].startswith('travel'):
            attack_type = 'travel'
        elif parts[1].startswith('workspace'):
            attack_type = 'workspace'
        else:
            attack_type = 'x'
        stats = extract_statistics(log_file)
        if stats:
            utility_count, utility_total, utility_ratio, security_count, security_total, security_ratio = stats
            result_utility_attackType[attack_type] = result_utility_attackType.get(attack_type, 0) + utility_count
            result_security_attackType[attack_type] = result_security_attackType.get(attack_type, 0) + security_count
            attack_cnt[attack_type] = attack_cnt.get(attack_type, 0) + security_total
            # print(stats)
            # results.append({
            #     'File': file_name,
            #     'Suite': suite,
            #     'Attack Type': attack_type,
            #     'Utility True Count': utility_count,
            #     'Utility Total': utility_total,
            #     'Utility Ratio': utility_ratio,
            #     'Security True Count': security_count,
            #     'Security Total': security_total,
            #     'Security Ratio': security_ratio
            # })

# Create DataFrame
df = pd.DataFrame(results)
print(f"Total files processed: {len(df)}")
df.head()
print("Utility True Count by Attack Type:")

# result_utility_attackType['type_ii'] = result_utility_attackType['type_ii_a'] + result_utility_attackType['type_ii_b']
# result_security_attackType['type_ii'] = result_security_attackType['type_ii_a'] + result_security_attackType['type_ii_b']
# attack_cnt['type_ii'] = attack_cnt['type_ii_a'] + attack_cnt['type_ii_b']
case_total = sum(attack_cnt.values())
utility_total = sum(result_utility_attackType.values())
security_total = sum(result_security_attackType.values())

for attack_type, count in result_security_attackType.items():
    result_security_ratio_attackType[attack_type] = count/attack_cnt[attack_type] if attack_cnt[attack_type] > 0 else 0.0
for attack_type, count in result_utility_attackType.items():
    result_utility_ratio_attackType[attack_type] = count/attack_cnt[attack_type] if attack_cnt[attack_type] > 0 else 0.0

for attack_type, count in result_utility_attackType.items():
    print(f"{attack_type}: {count}/ {attack_cnt[attack_type]} (Ratio: {result_utility_ratio_attackType[attack_type]:.4f})")
print(f"Overall Utility True Count: {utility_total}/ {case_total} (Ratio: {utility_total/case_total:.4f})")
print("\n")
print("Security True Count by Attack Type:")
for attack_type, count in result_security_attackType.items():
    print(f"{attack_type}: {count}/ {attack_cnt[attack_type]} (Ratio: {result_security_ratio_attackType[attack_type]:.4f})")  
print(f"Overall Security True Count: {security_total}/ {case_total} (Ratio: {security_total/case_total:.4f})") 

#!/usr/bin/env python3


Processing 4 log files...
Total files processed: 0
Utility True Count by Attack Type:
banking: 59/ 144 (Ratio: 0.4097)
slack: 34/ 105 (Ratio: 0.3238)
travel: 15/ 140 (Ratio: 0.1071)
workspace: 108/ 308 (Ratio: 0.3506)
Overall Utility True Count: 216/ 697 (Ratio: 0.3099)


Security True Count by Attack Type:
banking: 7/ 144 (Ratio: 0.0486)
slack: 17/ 105 (Ratio: 0.1619)
travel: 0/ 140 (Ratio: 0.0000)
workspace: 36/ 308 (Ratio: 0.1169)
Overall Security True Count: 60/ 697 (Ratio: 0.0861)
